# Creating tables and inserting data

In this exercise, you'll be creating two tables named `Series` and `SeriesMovies` and then populating them with data. You may remember these tables from one of the relational algebra exercises.

The basic syntax of the `CREATE TABLE` statement looks like this

```
  CREATE TABLE [<SCHEMA>.]<TABLENAME> (
    <COLUMN_1_NAME> <DATATYPE> NULL | NOT NULL [IDENTITY] [PRIMARY KEY],
    <COLUMN_2_NAME> <DATATYPE> NULL | NOT NULL [UNIQUE],
    ...,
    <COLUMN_2_NAME> <DATATYPE> NULL | NOT NULL [UNIQUE],
  )

```

The values contained between \<\> should be replaced by specific values, **not** including the \< and \> characters. Keywords like `IDENTITY` wrapped in \[ \] indicate **optional** portions of the syntax.

Here are some notes:

- An `IDENTITY` column will be generated automatically by SQL Server as an sequential integer value.
- Using the `PRIMARY KEY` designation will allow you to specify a **single** column as the primary key for the table; different syntax is needed to generate a multiple column primary key.
- The `UNIQUE` keyword can be used to tell the server it should enforce that the values in a column must be unique, without designating that column as the primary key. The `PRIMARY KEY` designation implies uniqueness.

Fill in the code cell below to create a table named **Series** in the **imdb** schema containing two columns:

- An `INT` column named `Id` whose values should be generated automatically and is designated as the primary key
- A `VARCHAR(64)` column named `SeriesName` whose values should be non-null and unique

The `INSERT` statement is used to insert one or more rows into a table. First, review the information in [Chapter 2 of the textbook on the insert statement](https://app.myeducator.com/reader/web/617gt/chapter02/w47sb/ub98v/). 

 Then write an `INSERT` statement in the cell below that creates a `Series` named _Star Wars_. Do not specify a value for the `Id` column, as the server will assign a value automatically.

Try running the previous cell multiple times.  You should see an error message whose contents hopefully make sense to you.  The name of the unique constraint will have been generated automatically by the server, but it includes the column name as part of the pattern.  You'll see how to give a specific name to a constraint like this soon!

You can insert multiple rows into a table at once by including comma-separated lists of parenthesized values, like this:

```
insert into Series [(col1, col2, ..., coln)] values (...), (...), (...)

```

Write a single `INSERT` statement below that creates 3 different `Series` tuples with names _Harry Potter_, _Rocky_, and _Die Hard_.

Write a query in the cell below that displays all attributes for all tuples in the `imdb.Series` table.

The order the tuples are displayed in might be surprising.  At least for me, they were displayed in order by `SeriesName` instead of by `Id`. You'll learn why this might have happened later in the semester when we talk about **indexes**.

Next, add another column to our `Series` table named `DateCreated`. To do this, you should use the `ALTER TABLE` command. Review the [textbook discussion of ALTER TABLE](https:\app.myeducator.com\reader\web\617gt\chapter05\bz065\ud0d5#yf1w6) and then write code in the cell below to add the `DateCreated` column to the `Series` table. 

- Use the `datetime` data type
- Do not allow null values 
- Specify that the value of the `getdate()` function should be used as the default value (see [SQL reference documentation on specifying defaults](https:\learn.microsoft.com\en-us\sql\relational-databases\tables\specify-default-values-for-columns?view=sql-server-ver16#create-table-t-sql))

Re-run your query that retrieved all attributes of all the tuples in the `imdb.Series` table and observe the values assigned to the `DateCreated` attribute for the existing tuples.

Surprise! The tuples are now most likely sorted by the `Id` column instead of the `Name` column. Again, you'll learn why this happened later in the semester (gotta have a good teaser to keep you coming back for more!)

## The SeriesMovies table

Next you'll create a table named **SeriesMovies** in the **imdb** schema that holds a many-to-many relationship between a `Series` and a `Movie`. You might think a one-to-many relationship is more appropriate, but perhaps we want to include `Star Wars Episode IV: A New Hope` in our _Star Wars_ series and also in an `Original Star Wars` series, for example.

Here's the list of columns you should include in `SeriesMovie`:

- `SeriesId`, a non-nullable integer that should **reference** the `Id` column in the `Series` table; deleting a `Series` tuple should cause the corresponding tuples in the `SeriesMovies` table to be deleted.
- `TitleId`, a non-nullable `char` field with size 10 that should reference the `TitleId` column in the `Movies` table; deleting a `Movies` tuple should cause the corresponding tuples in the `SeriesMovies` table to be deleted
- `SeriesOrder`, a non-nullable integer

The primary key for `SeriesMovie` should be set to the **combination** of the `SeriesId` and `TitleId` columns.

See [the SaleItem table definition in the textbook](https://app.myeducator.com/reader/web/617gt/chapter05/bz065/ud0d5/#l075u) for examples of

- how to create a primary key consisting of more than one column
- how to declare a foreign key that references another table, and to specify its behavior when the referenced tuple is deleted

After successfully creating the `SeriesMovies` table, press **CTRL-Shift-P** and search for _Refresh_, then select the _Refresh Intellisense Cache_ option to get Azure Data Studio to "know" about the existence of the `SeriesMovies` table.

Write a query below to select all attributes of all tuples from the `imdb.SeriesMovies` table. Hopefully auto-complete works for you after refreshing the Intellisense cache!

Now it's time to insert a movie into the `SeriesMovies` table.  First write two `SELECT` statements in the code cell below to find 

- the `Series.Id` value for the series named _Star Wars_
- the `Movies.TitleId` value for the movie with `Original Title` _Star Wars_

Write an `INSERT` statement that inserts the appropriate values into the `MovieSeries` table, using **1** as the value for the `SeriesOrder` attribute.

Write a `SELECT` statement that shows the `SeriesName`, `Title`, and `OriginalTitle` attributes for all the movies in the series named _Star Wars_.

It's a hassle to look up the `Id` and `TitleId` attribute values manually. You can instead use a `SELECT` statement to look up the values for these attributes. Here's an example you should execute to add _The Empire Strikes Back_ to the _Star Wars_ series.

In [ ]:
insert into imdb.SeriesMovies
select Id, TitleId, 2
from imdb.Series, imdb.Movies
where SeriesName = 'Star Wars'
  and Title like '%Empire Strikes Back%'

There isn't a `JOIN` in the `FROM` clause. This means you are taking the **cartesian product** of the tuples identified from the `Series` and `Movies` tables. Since there's only 1 tuple from each, a single tuple is inserted.

Re-run your query listing the movies in the _Star Wars_ series to verify you get both movies. Add an `ORDER BY` clause to ensure they are displayed in the correct order.

In the code cell below, complete the starter query as instructed so it adds _Return of the Jedi_ to the _Star Wars_ series with the appropriate value for the `SeriesOrder` attribute

In [ ]:
with LargestSeriesOrder (MaxSeriesOrder) as (
    --  Replace the query below with one to find the largest SeriesOrder value for the series named Star Wars
    select 3
)
insert into imdb.SeriesMovies
select Id, TitleId, MaxSeriesOrder+1
from imdb.Series, imdb.Movies, LargestSeriesOrder
where SeriesName = 'Star Wars'
  and Title like '%Return of the Jedi%'

Re-run your query listing the movies in the _Star Wars_ series to make sure you see all 3 of the original series movies.

Then, review the [textbook documentation on update](https://app.myeducator.com/reader/web/617gt/w47sb#gn9ws) and write an `UPDATE` statement to change the name of the _Star Wars_ series to _Original Star Wars_.

Now write and execute an `INSERT` state to create another `Series` named _Full Star Wars_ which will eventually contain all 9 of the Star Wars movies.

Write an `INSERT` statement that adds the tuples in `SeriesMovies` for the _Original Star Wars_ movies as tuples in the _Full Star Wars_ series. For now, keep the same values for the `SeriesOrder` column values. Don't hard-code the value of the `SeriesId` attribute; look it up from `Series` instead. 

Hint:  you can use a **query** to provide the value for an attribute <span style="color: var(--vscode-foreground);">as long as the parenthesized query returns a</span> **single value**<span style="color: var(--vscode-foreground);">, like this:</span>

```
select (select column from table where criteria), column1, column2
from table

```

 I used this to look up the `Id` attribute value for the _Full Star Wars_ series.

We need to make way for the prequels.  Write an **UPDATE** statement to add 3 to the `SeriesOrder` attribute values associated with the _Full Star Wars_ series.  You'll need to write a **subquery** to find the correct value for the `SeriesId` attribute, something like this:

````
update ...
set ....
where SeriesId = (
    --  subquery goes here
)
````

And, write a query that display the `SeriesName`, `Title`, and `SeriesOrder` for **all** the existing `Series` (including _Harry Potter_, _Die Hard_, and _Rocky_).

Sort the results first by `Id`, then by `SeriesOrder`.

The results should look like this:

| SeriesName | Title | SeriesOrder |
| --- | --- | --- |
| Original Star Wars | Star Wars: Episode IV - A New Hope | 1 |
| Original Star Wars | Star Wars: Episode V - The Empire Strikes Back | 2 |
| Original Star Wars | Star Wars: Episode VI - Return of the Jedi | 3 |
| Harry Potter | _NULL_ | _NULL_ |
| Rocky | _NULL_ | _NULL_ |
| Die Hard | _NULL_ | _NULL_ |
| Original Star Wars | Star Wars: Episode IV - A New Hope | 4 |
| Original Star Wars | Star Wars: Episode V - The Empire Strikes Back | 5 |
| Original Star Wars | Star Wars: Episode VI - Return of the Jedi | 6 |

Your last task for this exercise will be to insert the appropriate entries into the _Harry Potter_ and _Rocky_ series.  You can use the `Rank()` function to make this quick. Examine and execute the cell below to insert the _Harry Potter_ movies.  There should be 10 movies inserted into the `SeriesMovies` table.

In [ ]:
with HarryPotter(Id) as (
    select Id 
    from imdb.Series
    where SeriesName = 'Harry Potter'
)
insert into imdb.SeriesMovies (SeriesId, TitleId, SeriesOrder)
select Id, TitleId, rank() over (order by YearReleased)
from imdb.Movies, HarryPotter
where Title like 'Harry Potter%'

Execute the cell above that lists all the movies associated with series to verify the _Harry Potter_ movies were inserted correctly.

Then use a similar technique to insert movies into the _Rocky_ series, using our standard definition of a Rocky movie:  a movie _Sylvester Stallone_ played a role in whose title starts with _Rocky_.

I used my `ActorMovies` view to make it easier to identify the 6 Rocky movies.

Once you've executed the query to fill in the _Rocky_ series with movies, execute the cell above that lists all the movies associated with series to verify the _Rocky_ movies were inserted correctly.